In [2]:
import sys
sys.path.append('../../../build/python')

import os # Configure which GPU
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')


import mitsuba as mi
import drjit as dr
import tensorflow as tf
import torch
%load_ext autoreload
%autoreload 2
sys.path.append('../drjit')

from interop import wrap, to_drjit, from_drjit, flatten

I0000 00:00:1728982580.327655  188118 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728982580.366881  188118 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728982580.369725  188118 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [17]:
@wrap('torch', 'drjit')
def test_fn(x, y, z):
    return x*2, y+1, ~z

dt = tf.float32
x = tf.constant([0, 1, 2], dtype=dt)
y = tf.cast(x, dtype=tf.int32)
if True:
    z = y > 0
else:
    z = y

a, b, c = test_fn(x, y, z)
assert tf.reduce_all(a == x*2)  and tf.reduce_all(b == y+1) and tf.reduce_all(c == ~z)
#assert torch.all(a == x*2) and torch.all(b == y + 1) and torch.all(c == ~z)
with tf.GradientTape() as tape:
    tape.watch(x)
    a, b, c = test_fn(x, y, z)
grad = tape.gradient(a, x, output_gradients=tf.constant([10, 20, 30], dtype=dt))
assert tf.reduce_all(grad == tf.constant([20, 40, 60], dtype=dt))

In [55]:
def fun(x):
    return x
tf.nest.map_structure(fun, [x, y])

[[0, 1, 2], <__main__.MyClass at 0x7f81fc191070>]

In [58]:
class MyClass:
        pass

@wrap('drjit', 'tf')
def test_fn(x, y):
    return x, y

x = dr.arange(dr.cuda.ad.Float32, 3)
dr.enable_grad(x)

y = MyClass()

a, b = test_fn(x, y)
assert dr.all(a == x) and b is y

a.grad = [10, 20, 30]
dr.backward_to(x)
assert dr.all(x.grad == [10, 20, 30])

In [91]:
@wrap('torch', 'drjit')
def test_fn(x):
        return {
            123:(x[0]["hello"] + 2*x[1]["world"][0])
        }
dt = torch.float32
x = torch.tensor([1, 2, 3], dtype=dt, requires_grad=True)
y = torch.tensor([4, 5, 6], dtype=dt, requires_grad=True)
xt = [
    { 'hello' : x },
    { 'world' : (y,) }
]
rt = test_fn(xt)
r = rt[123]

torch.autograd.backward(r, torch.tensor([100, 200, 300], dtype=dt))
assert torch.all(x.grad == torch.tensor([100, 200, 300], dtype=dt))
assert torch.all(y.grad == torch.tensor([200, 400, 600], dtype=dt))

[[9, 12, 15]] ([100, 200, 300],)


In [99]:
@wrap('tf', 'drjit')
def test_fn(x):
        return {
            123:(x[0]["hello"] + 2*x[1]["world"][0])
        }

dt = tf.float32
x = tf.constant([1, 2, 3], dtype=dt)
y = tf.constant([4, 5, 6], dtype=dt)
with tf.GradientTape() as tape:
    tape.watch([x, y])
    xt = [
        { 'hello' : x },
        { 'world' : (y,) }
    ]
    rt = test_fn(xt)
    r = rt[123]
grad = tape.gradient(r, [x, y], output_gradients=tf.constant([100, 200, 300], dtype=dt))
assert tf.reduce_all(grad[0] == tf.constant([100, 200, 300], dtype=dt))
assert tf.reduce_all(grad[1] == tf.constant([200, 400, 600], dtype=dt))

[9, 12, 15]
[100, 200, 300]


In [142]:
@wrap('tf', 'drjit')
def test_fn(x):
    return x * 2

is_diff = True
scalar_deriv = False

dt = tf.float32
x = tf.cast(tf.range(3), dt)
y = test_fn(x)
assert tf.reduce_all(y == x * 2)
if is_diff:
    if scalar_deriv:
        with tf.GradientTape() as tape:
            tape.watch(x)
            out = test_fn(x)
        grad = tape.gradient(out, x)
        assert tf.reduce_all(grad == tf.constant([2, 2, 2], dtype=dt))
    else:
        with tf.GradientTape() as tape:
            tape.watch(x)
            out = test_fn(x)
        grad = tape.gradient(out, x, output_gradients=tf.constant([10, 20, 30], dtype=dt))
        assert tf.reduce_all(grad == tf.constant([20, 40, 60], dtype=dt))

[0, 2, 4]
1


In [148]:
@wrap('tf', 'drjit')
def test_fn(x, y):
    return x + y, y, x

dt = tf.float32
x = tf.constant([0, 1, 2], dtype=dt)
y = tf.constant([4], dtype=dt)
a, b, c = test_fn(x, y)

assert tf.reduce_all(a == tf.constant([4, 5, 6], dtype=dt))
assert tf.reduce_all(b == tf.constant([4], dtype=dt))
assert tf.reduce_all(c == tf.constant([0, 1, 2], dtype=dt))

with tf.GradientTape() as tape:
    tape.watch([x, y])
    a, b, c = test_fn(x, y)
grad = tape.gradient([a, b, c], [x, y],
                    output_gradients=[tf.constant([10, 20, 30], dtype=dt),
                                    tf.constant([40], dtype=dt),
                                    tf.constant([50, 60, 70], dtype=dt)])

In [147]:
import torch.autograd.forward_ad as fwd_ad

@wrap('torch', 'drjit')
def test_fn(x, y):
    return x + y, y + 1, x + 1

dt = torch.float32
x = torch.arange(3, dtype=dt, requires_grad=True)
y = torch.tensor([4], dtype=dt, requires_grad=True)
xd = torch.tensor([10, 20, 30], dtype=dt)
yd = torch.tensor([40], dtype=dt)

with fwd_ad.dual_level():
    x = fwd_ad.make_dual(x, xd)
    y = fwd_ad.make_dual(y, yd)
    a, b, c = test_fn(x, y)

    a, ad = fwd_ad.unpack_dual(a)
    b, bd = fwd_ad.unpack_dual(b)
    c, cd = fwd_ad.unpack_dual(c)

    print(ad)
    print(bd)
    print(cd)

tensor([50., 60., 70.])
tensor([40.])
tensor([10., 20., 30.])


In [42]:
import tensorflow as tf

# Define a custom operation with multiple outputs and custom gradient
@tf.custom_gradient
def test_fn(x, y):
    # Forward pass: compute the outputs
    result1 = x + y
    result2 = y + 1
    result3 = x + 1

    def grad(upstream1, upstream2, upstream3):
        print(upstream1, upstream2, upstream3)
        # Gradients of the output with respect to inputs x and y
        grad_x = upstream1 + upstream3  # Derivative of result1 and result3 w.r.t x
        grad_y = upstream1 + upstream2  # Derivative of result1 and result2 w.r.t y
        return grad_x, grad_y

    return (result1, result2, result3), grad

# Example usage of the custom operation
x = tf.cast([0,1,2], dtype=dt)
y = tf.constant([4], dtype=dt)

with tf.GradientTape() as tape:
    tape.watch([x, y])
    res1, res2, res3 = test_fn(x, y)
    loss = res1 + res2 + res3  # Example loss function using all outputs
    print(loss)

# Compute gradients with respect to inputs x and y
grads = tape.gradient(loss, [x, y])

# Print results
print('Result 1:', res1.numpy())
print('Result 2:', res2.numpy())
print('Result 3:', res3.numpy())
print('Gradient w.r.t x:', grads[0].numpy())
print('Gradient w.r.t y:', grads[1].numpy())

tf.Tensor([10. 12. 14.], shape=(3,), dtype=float32)
tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32) tf.Tensor([3.], shape=(1,), dtype=float32) tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)
Result 1: [4. 5. 6.]
Result 2: [5.]
Result 3: [1. 2. 3.]
Gradient w.r.t x: [2. 2. 2.]
Gradient w.r.t y: [4. 4. 4.]


In [84]:
import tensorflow as tf

# Define a custom operation with multiple outputs and custom gradient
@tf.custom_gradient
def test_fn(x, y):
    # Forward pass: compute the outputs
    result1 = x + y
    result2 = y + 1
    result3 = x + 1

    def grad(dresult1, dresult2, dresult3):
        # Gradients of the outputs with respect to inputs x and y
        # Each dresult is a tensor representing the upstream gradient for each output

        # Compute gradients for x and y
        grad_x = dresult1 + dresult3  # Derivative of result1 and result3 w.r.t x
        grad_y = dresult1 + dresult2  # Derivative of result1 and result2 w.r.t y

        # Broadcast gradients to match input shapes
        print(grad_x)
        print(grad_y)
        grad_x = tf.broadcast_to(grad_x, tf.shape(x))
        grad_y = tf.broadcast_to(grad_y, tf.shape(y))

        return grad_x, grad_y

    return (result1, result2, result3), grad

# Example usage with ForwardAccumulator for forward-mode autodiff
dt = tf.float32
x = tf.cast([0, 1, 2], dtype=dt)  # Shape (3,)
y = tf.constant([4], dtype=dt)    # Shape (1,)
xd = tf.constant([10, 20, 30], dtype=dt)  # Shape (3,)
yd = tf.constant([40], dtype=dt)          # Shape (1,)

with tf.autodiff.ForwardAccumulator(
            primals=[x, y],
            tangents=[xd, yd]
        ) as acc:
    out = test_fn(x, y)

grad_out = acc.jvp(out)

# Print results of forward-mode gradients
print("Gradient of first output:", grad_out[0].numpy())
print("Gradient of second output:", grad_out[1].numpy())
print("Gradient of third output:", grad_out[2].numpy())

tf.Tensor([2. 2. 2.], shape=(3,), dtype=float32)
tf.Tensor([2. 2. 2.], shape=(3,), dtype=float32)


InvalidArgumentError: {{function_node __wrapped__BroadcastTo_device_/job:localhost/replica:0/task:0/device:GPU:0}} Unable to broadcast tensor of shape [3] to tensor of shape [1] [Op:BroadcastTo] name: 

In [127]:
tf.rank(x) - tf.rank(y)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [124]:
x = tf.random.normal(shape=[10,1,20], dtype=tf.float32)
y = tf.random.normal(shape=[20], dtype=tf.float32)
z = tf.broadcast_to(y, x.shape)


def get_broadcast_info(original_shape, target_shape):
    padded_shape = [1] * (len(target_shape) - len(original_shape)) + list(original_shape)
    added_dims = [i for i, (orig_dim, target_dim) in enumerate(zip(padded_shape, target_shape)) if orig_dim == 1 and target_dim != 1]
    return added_dims


get_broadcast_info(y.shape, z.shape)







[0]

In [429]:
@tf.custom_gradient
def fn(x):
    y = tf.stop_gradient(x**3)
    @tf.custom_gradient
    def grad(dy):
        grads = tf.stop_gradient(3*x**2*dy)
        def grad(dz):
            print(dz)
            return dz*grads
        return grads, grad
    return y, grad

def fn2(x):
    return x**3

x = tf.cast([0, 1, 2, 3], dtype=dt)
xd = tf.constant([10, 20, 30, 12], dtype=dt)
with tf.autodiff.ForwardAccumulator(
            primals=x,
            tangents=xd
        ) as acc:
    out = fn(x)
grad_out = acc.jvp(out)
print(grad_out)

with tf.autodiff.ForwardAccumulator(
            primals=x,
            tangents=xd
        ) as acc:
    out = fn2(x)
grad_out = acc.jvp(out)
print(grad_out)

tf.Tensor([10. 20. 30. 12.], shape=(4,), dtype=float32)
tf.Tensor([  0.  60. 360. 324.], shape=(4,), dtype=float32)
tf.Tensor([  0.  60. 360. 324.], shape=(4,), dtype=float32)


In [381]:
import numpy as np

@tf.custom_gradient
def test_fn(x):
    y = tf.stop_gradient(2*x)
    def grad(dy):
        return 2*d
    return y, grad
    return y, grad

def test_fn2(x):
    return 2*x

dt = tf.float32
x = tf.cast([0, 1, 2], dtype=dt)
xd = tf.constant([10, 20, 30], dtype=dt)
with tf.autodiff.ForwardAccumulator(
            primals=x,
            tangents=xd
        ) as acc:
    out = test_fn(x)
grad_out = acc.jvp(out)
print(grad_out)


None


In [404]:
a = [1,2,3]
b = [4,5,6]

for i in zip(a,b):
    print(i)

(1, 4)
(2, 5)
(3, 6)


In [431]:
@tf.custom_gradient
def test_fn(x, y):
    results = x + y, y + 1, x + 1
    results = [tf.stop_gradient(el) for el in results]

    @tf.custom_gradient
    def grad(*z):
        g = z[0]*tf.ones_like(x)+z[2]*tf.ones_like(x), tf.reduce_sum(tf.ones_like(x)*z[0]) + z[1]
        g = [tf.stop_gradient(el) for el in g]

        def grad2(*s):
            res = 
            return res
        return g, grad2

    return results, grad

def test_fn2(x, y):
    results = x + y, y + 1, x + 1
    return results

@wrap('tf', 'drjit')
def test_fn3(x, y):
    results = x + y, y + 1, x + 1
    return results

for f in [test_fn, test_fn2]:

    dt = tf.float32
    x = tf.cast([0, 1, 2], dtype=dt)
    y = tf.constant([4], dtype=dt)
    xd = tf.constant([10, 20, 30], dtype=dt)
    yd = tf.constant([40], dtype=dt)
    with tf.autodiff.ForwardAccumulator(
                primals=[x, y],
                tangents=[xd, yd]
            ) as acc:
        out = f(x, y)
    grad_out = acc.jvp(out)
    print(grad_out[0])
    print(grad_out[1])
    print(grad_out[2])



(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([10., 20., 30.], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.], dtype=float32)>)


ValueError: custom_gradient function expected to return 3 gradients, but returned 2 instead.

In [52]:
import torch.autograd.forward_ad as fa

x_primal = torch.randn(10, 10, dtype=torch.float32)
x_tangent = torch.randn(10, 10, dtype=torch.float32)

y_primal = torch.randn(10, 10, dtype=torch.float32)
y_tangent = torch.zeros_like(y_primal)

def fn(x, y):
    return x ** 2 + y ** 2

with fa.dual_level():
    x_dual = fa.make_dual(x_primal, x_tangent)
    y_dual = fa.make_dual(y_primal, y_tangent)
    z_dual = fn(x_dual, y_dual)
    z_tangent = fa.unpack_dual(z_dual).tangent


x_primal_tf = tf.constant(x_primal.numpy())
x_tangent_tf = tf.constant(x_tangent.numpy())
y_primal_tf = tf.constant(y_primal.numpy())
y_tangent_tf = tf.constant(y_tangent.numpy())
with tf.autodiff.ForwardAccumulator(primals=[x_primal_tf, y_primal_tf],
                                    tangents=[x_tangent_tf, y_tangent_tf]) as acc:
    z_primal_tf = fn(x_primal_tf, y_primal_tf)
    z_tangent_tf = acc.jvp(z_primal_tf)


z_tangent.numpy() == z_tangent_tf.numpy()



import numpy as np
print(np.max(np.abs(z_tangent_tf - z_tangent)))
print(np.max(np.abs(z_primal_tf - z_dual)))


0.0
4.7683716e-07


In [72]:
tf.convert_to_tensor(8)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [ ]:
def f(x=x):


In [79]:
@wrap('tf', 'drjit')
def test_fn(x, y):
        return x + y#,  y + 1, x + 1
dt = tf.float32
x = tf.cast(tf.range(3), dtype=dt)
y = tf.constant([4], dtype=dt)
xd = tf.constant([10, 20, 30], dtype=dt)
yd = tf.constant([40], dtype=dt)
with tf.autodiff.ForwardAccumulator(
            primals=[x, y],
            tangents=[xd, yd]
        ) as acc:
    out = test_fn(x, y)
grad_out = acc.jvp(out)
assert tf.reduce_all(grad_out[0] == [50, 60, 70])
assert tf.reduce_all(grad_out[1] == [40])
assert tf.reduce_all(grad_out[2] == [10, 20, 30])

TypeError: 'NoneType' object is not subscriptable

In [112]:
x = {'x' : tf.constant([1, 2, 4], tf.float32)}
y = tf.constant([1, 2, 4], tf.float32)
def f(x, y):
    return 2*x['x'] + y**2

with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    tape.watch(y)
    z = f(x, y)
grad_x = tape.gradient(z, x)
grad_y = tape.gradient(z, y)
print(grad_x)
print(grad_y)

{'x': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 2., 2.], dtype=float32)>}
tf.Tensor([2. 4. 8.], shape=(3,), dtype=float32)


In [124]:
@wrap('drjit', 'torch')
def test_fn(x):
    return {
        123:(x[0]["hello"] + 2*x[1]["world"][0])
    }
x = dr.llvm.ad.Float(1, 2, 3)
y = dr.llvm.ad.Float(4, 5, 6)
dr.enable_grad(x, y)
xt = [
    { 'hello' : x },
    { 'world' : (y,) }
]
rt = test_fn(xt)
r = rt[123]

r.grad = [100, 200, 300]
dr.backward_to(x, y)
#print(y.grad)
assert dr.all(x.grad == [100, 200, 300])
assert dr.all(y.grad == [200, 400, 600])

([{'hello': tensor([100., 200., 300.])}, {'world': (tensor([200., 400., 600.]),)}],)


In [129]:
@wrap('drjit', 'tf')
def test_fn(x):
    return {
        123:(x[0]["hello"] + 2*x[1]["world"][0])
    }
x = dr.llvm.ad.Float(1, 2, 3)
y = dr.llvm.ad.Float(4, 5, 6)
dr.enable_grad(x, y)
xt = [
    { 'hello' : x },
    { 'world' : (y,) }
]
rt = test_fn(xt)
r = rt[123]

r.grad = [100, 200, 300]
dr.backward_to(x, y)
#print(y.grad)
assert dr.all(x.grad == [100, 200, 300])
assert dr.all(y.grad == [200, 400, 600])

([{'hello': <tf.Tensor: shape=(3,), dtype=float32, numpy=array([100., 200., 300.], dtype=float32)>}, {'world': (<tf.Tensor: shape=(3,), dtype=float32, numpy=array([200., 400., 600.], dtype=float32)>,)}],)


In [134]:
## Forward-mode AD
@wrap('drjit', 'tf')
def test_fn(x, y, z):
    return x*2, y, z

x = dr.arange(dr.llvm.ad.Float, 3)
dr.enable_grad(x)

a, b, c = test_fn(x, 4, 5.0)
assert dr.all(a == x*2) and dr.all(b == 4) and dr.all(c == 5)

a.grad = [10, 20, 30]
dr.backward_to(x)

assert dr.all(x.grad == [20, 40, 60])





ValueError: Passed in object 4 of type 'int', not tf.Tensor or tf.Variable or ExtensionType.

In [136]:
tf.reduce_all

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>

In [63]:
# Test for a single tensor input

@wrap('tf', 'drjit')
def dr_func(input):
    return dr.power(input, 2)

x = tf.constant([1,2,3,4], tf.float32)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = dr_func(x)
grad = tape.gradient(y, x)
print("x:", x)
print("y:", y)
print("Gradient:", grad)  


x: tf.Tensor([1. 2. 3. 4.], shape=(4,), dtype=float32)
y: tf.Tensor([ 1.  4.  9. 16.], shape=(4,), dtype=float32)
Gradient: tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)


In [64]:
# Test with a list of tensors

@wrap('tf', 'drjit')
def dr_func(inputs):
    result = 0
    for i, input in enumerate(inputs):
        result += dr.power(input, i+1)
    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
vars = [x1, x2, x3]
with tf.GradientTape() as tape:
    tape.watch(vars)
    result = dr_func(vars)
grad = tape.gradient(result, vars)
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)

Result: tf.Tensor([ 3. 14. 39. 84.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)
tf.Tensor([ 3. 12. 27. 48.], shape=(4,), dtype=float32)


In [52]:
# Test with a nested list of tensors
# Test with a list of tensors

@wrap('tf', 'drjit')
def dr_func(inputs):
    inputs = sum(inputs, [])
    result = 0
    for i, input in enumerate(inputs):
        result += dr.power(input, i+1)
    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x3 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x4 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
vars = [[x1, x2], [x3, x4]]
with tf.GradientTape() as tape:
    tape.watch(vars)
    result = dr_func(vars)
grad = tape.gradient(result, [x1, [x2, x3, x4]])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([  4.  30. 120. 340.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2., 4., 6., 8.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 3., 12., 27., 48.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([  4.,  32., 108., 256.], dtype=float32)>]


In [66]:
# Test with multiple arguments
@wrap('tf', 'drjit')
def dr_func(x1, x2):
    return dr.power(x1, 1) + dr.power(x2, 3)

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch([x1, x2])
    result = dr_func(x1, x2)
grad = tape.gradient(result, [x1, x2])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([ 2. 10. 30. 68.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
tf.Tensor([ 3. 12. 27. 48.], shape=(4,), dtype=float32)


In [67]:
# Test with keyword argument
@wrap('tf', 'drjit')
def dr_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += dr.power(x, i+1)

    for i, x in enumerate(kwargs.values()):
        result += 4*x

    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([2, 3, 4, 5], dtype=tf.float32)
x3 = tf.constant([3, 4, 4, 5], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch([x1, x2, x3])
    result = dr_func([x1, x2], x3=x3)
grad = tape.gradient(result, [x1, x2, x3])
print(grad)

[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>, <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 4.,  6.,  8., 10.], dtype=float32)>, None]


In [70]:
# Test with dict argument
@wrap('tf', 'drjit')
def dr_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += dr.power(x, i+1)

    for i, x in enumerate(args[1].values()):
        result += 4*(i+1)*x

    return result

def tf_func(*args, **kwargs):
    result = 0
    for i, x in enumerate(args[0]):
        result += tf.pow(x, i+1)

    for i, x in enumerate(args[1].values()):
        result += 4*(i+1)*x

    return result

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([2, 3, 4, 5], dtype=tf.float32)
x3 = tf.constant([3, 4, 4, 5], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch([x1, x2, x3])
    result = dr_func(x1, {'x2' : x2, 'x3': x3})
grad = tape.gradient(result, [x2])
print(result)
for g in grad:
    print(g)

tf.Tensor([320. 332. 336. 348.], shape=(4,), dtype=float32)
tf.Tensor([4. 4. 4. 4.], shape=(4,), dtype=float32)


In [76]:
from tensorflow.autodiff import ForwardAccumulator

In [69]:
# Test with a non differentiable input
@wrap('tf', 'drjit')
def dr_func(x1, x2):
    return dr.power(x1, 1) + dr.power(x2, 1)

def tf_func(x1, x2):
    return tf.pow(x1, 1) + tf.cast(tf.pow(x2, 1), x1.dtype)

x1 = tf.constant([1, 2, 3, 4], dtype=tf.float32)
x2 = tf.constant([1, 2, 3, 4], dtype=tf.int32)
with tf.GradientTape() as tape:
    tape.watch([x1, x2])
    result = dr_func(x1, x2)
grad = tape.gradient(result, [x1, x2])
print("Result:", result)
print("Gradient:")
for g in grad:
    print(g)


Result: tf.Tensor([2. 4. 6. 8.], shape=(4,), dtype=float32)
Gradient:
tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32)
None


In [6]:
import tensorflow as tf
tf.constant([1,2,3,4], tf.float32)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 2., 3., 4.], dtype=float32)>

In [17]:
import drjit as dr
from drjit.cuda.ad import Float
from drjit.cuda.ad import Int32

t = Int32
is_diff = False

@wrap('drjit', 'torch')
def test_fn(x):
    return x * 2

x = dr.arange(t, 3)
dr.enable_grad(x)
y = test_fn(x)
assert dr.all(y == [0, 2, 4])


if is_diff:
    y.grad = [10, 20, 30]
    dr.backward_to(x)
    assert dr.all(x.grad == [20, 40, 60])
else:
    assert not dr.grad_enabled(y)






In [35]:
from interop import from_drjit, to_drjit

t = dr.llvm.ad.Float
x = dr.arange(t, 3)
z, _ = from_drjit(x, 'tf', True)
y = z*2
z_ = to_drjit(y, 'tf')


In [12]:
import drjit as dr


@wrap('drjit', 'tf')
def test_fn(x):
    y = x*2
    print(y.device)
    return y

t = dr.cuda.ad.Float
x = dr.arange(t, 3)
y = test_fn(x)



/job:localhost/replica:0/task:0/device:GPU:0
